In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import time

# Read in data

In [3]:
bz = pd.read_csv('yelp_dataset/restaurant.csv',index_col='Unnamed: 0')
rv = pd.read_csv('yelp_dataset/reviews_filtered.csv',index_col='Unnamed: 0')
us = pd.read_csv('yelp_dataset/user_filtered.csv',index_col='Unnamed: 0')

# bz --> rv --> us

In [4]:
# bz1 = bz.sample(frac=0.1)
# rv1 = bz1[['business_id']].merge(rv, how='left',on='business_id')
# us1 = rv1[['user_id']].merge(rv1, how='left',on='user_id')

In [5]:
df = rv.groupby(['business_id','user_id']).agg({'stars':'mean'}).reset_index()

In [6]:
df.head()

,business_id,user_id,stars
0,--9e1ONYQuAa-CB_Rrw7Tw,---1lKK3aKOuomHnwAkAow,4.0
1,--9e1ONYQuAa-CB_Rrw7Tw,-11nMat3F0DgsxT30hn8zA,5.0
2,--9e1ONYQuAa-CB_Rrw7Tw,-3fMaL_ck0wzEsTZyz-mqA,5.0
3,--9e1ONYQuAa-CB_Rrw7Tw,-47EprTYhe9IbAvplGIcpA,5.0
4,--9e1ONYQuAa-CB_Rrw7Tw,-7cPERHtAupd1Xq53nmhXg,5.0


# MF Attempt

In [ ]:
msk = np.random.rand(len(df)) < 0.7
df_train = df[msk]
user_ind = [x-1 for x in df_train['user_id'].index]
item_ind = [x-1 for x in df_train['business_id'].index]
stars = df_train['stars'].values

In [ ]:
df_train['user_id'].nunique()

In [ ]:
df_train['business_id'].nunique()

In [ ]:
# variables
feature_len = 10
U = tf.Variable(initial_value=tf.truncated_normal([373483,feature_len]), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([feature_len,8848]), name='items')
result = tf.matmul(U, P)
result_flatten = tf.reshape(result, [-1])

In [ ]:
R = tf.gather(result_flatten, user_ind * tf.shape(result)[1] + 
              item_ind, name='extracting_user_stars')

In [ ]:
# cost function
diff_op = tf.subtract(R, stars, name='training_diff')
diff_op_squared = tf.abs(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

In [ ]:
# regularization
lda = tf.constant(.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm'), 
   tf.reduce_sum(tf.abs(P, name='item_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

cost = tf.add(base_cost, regularizer)

In [ ]:
# optimizer
lr = tf.constant(.001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [ ]:
# execution
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
for i in xrange(500):
    sess.run(training_step)


In [ ]:
# testing
u, p, r = df[['user', 'item', 'rate']].values[0]
rhat = tf.gather(tf.gather(result, u-1), p-1)
print "rating for user " + str(u) + " for item " + str(p) + 
      " is " + str(r) + " and our prediction is: " + str(sess.run(rhat))

In [ ]:
# calculate accuracy
df_test = df[~msk]
user_indecies_test = [x-1 for x in df_test.user.values]
item_indecies_test = [x-1 for x in df_test.item.values]
rates_test = df_test.rate.values

# accuracy
R_test = tf.gather(result_flatten, user_indecies_test * tf.shape(result)[1] + item_indecies_test, name='extracting_user_rate_test')
diff_op_test = tf.sub(R_test, rates_test, name='test_diff')
diff_op_squared_test = tf.abs(diff_op, name="squared_difference_test")

cost_test = tf.div(tf.reduce_sum(tf.square(diff_op_squared_test, name="squared_difference_test"), name="sum_squared_error_test"), df_test.shape[0] * 2, name="average_error")
print sess.run(cost_test)

# CF Attempt

In [ ]:
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

from surprise import SVD
from surprise import Dataset
from surprise import NormalPredictor
from surprise import Reader
from surprise import accuracy
from surprise import similarities
from surprise import KNNBasic

In [ ]:
# bz1 = bz.sample(frac=0.1)
# rv1 = bz1[['business_id']].merge(rv, how='left',on='business_id')
# us1 = rv1[['user_id']].merge(rv1, how='left',on='user_id')

In [ ]:
# reader, the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars']], reader)

In [ ]:
cross_validate(NormalPredictor(), data, cv=5)

In [ ]:
# split the data into test and training
trainset, testset = train_test_split(data, test_size=0.2)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

In [ ]:
cross_validate(algo, data, measures=['rmse', 'mae'], cv=5,
               return_train_measures=False, n_jobs=1,
               pre_dispatch='2*n_jobs', verbose=False)

In [ ]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

#### cosine similarity

In [ ]:
sim_options = {'name': 'cosine'}
algo2 = KNNBasic(sim_options=sim_options)

algo2.fit(trainset)
similarities = algo2.test(testset)

accuracy.rmse(similarities)

In [ ]:
cross_validate(algo2, data, measures=['rmse', 'mae'], cv=2,
               return_train_measures=False, n_jobs=1,
               pre_dispatch='2*n_jobs', verbose=False)